# AutoML

In [1]:
import numpy as np
import tensorflow as tf

In [2]:
pip install autokeras

     |████████████████████████████████| 174kB 23.0MB/s 
     |████████████████████████████████| 71kB 9.8MB/s 
  Created wheel for keras-tuner: filename=keras_tuner-1.0.2-cp36-none-any.whl size=78939 sha256=6de8ce93fd489a32c65646de3c4565e1bc73fb5b2e88747003ec8681660dc5c1
  Stored in directory: /root/.cache/pip/wheels/bb/a1/8a/7c3de0efb3707a1701b36ebbfdbc4e67aedf6d4943a1f463d6
  Created wheel for terminaltables: filename=terminaltables-3.1.0-cp36-none-any.whl size=15358 sha256=3650ec1cb47c18478933ae3b9e0e2f1b0a5bcfd0ddee1007be3a6475d21b4b9b
  Stored in directory: /root/.cache/pip/wheels/30/6b/50/6c75775b681fb36cdfac7f19799888ef9d8813aff9e379663e
Successfully built keras-tuner terminaltables


## 모델 정의, 설정, 학습

In [6]:
# Load data set
from tensorflow.keras.datasets import fashion_mnist

data_set_name = "fashion_mnist"
(x_train, y_train), (x_test, y_test) = fashion_mnist.load_data()
x_train = x_train.reshape(x_train.shape + (1,))
x_test = x_test.reshape(x_test.shape + (1,))

x_train.shape, x_test.shape

((60000, 28, 28, 1), (10000, 28, 28, 1))

In [23]:
# Create Auto-Keras model
import autokeras as ak

# Initialize the image classifier.
clf = ak.ImageClassifier(max_trials=1, overwrite=True)

# Feed the image classifier with training data.
clf.fit(x_train, y_train, epochs=30)

# Export as a Keras Model.
model = clf.export_model()

print(type(model))  # <class 'tensorflow.python.keras.engine.training.Model'>

try:
    model.save("model_autokeras", save_format="tf")
except:
    model.save("model_autokeras.h5")

Trial 1 Complete [00h 02m 46s]
val_loss: 0.2218191921710968

Best val_loss So Far: 0.2218191921710968
Total elapsed time: 00h 02m 46s
INFO:tensorflow:Oracle triggered exit
Epoch 1/30
1875/1875 [==============================] - 7s 3ms/step - loss: 0.5436 - accuracy: 0.8061
Epoch 2/30
1875/1875 [==============================] - 7s 4ms/step - loss: 0.3147 - accuracy: 0.8859
Epoch 3/30
1875/1875 [==============================] - 7s 4ms/step - loss: 0.2803 - accuracy: 0.8989
Epoch 4/30
1875/1875 [==============================] - 7s 4ms/step - loss: 0.2574 - accuracy: 0.9069
Epoch 5/30
1875/1875 [==============================] - 7s 3ms/step - loss: 0.2428 - accuracy: 0.9107
Epoch 6/30
1875/1875 [==============================] - 6s 3ms/step - loss: 0.2288 - accuracy: 0.9155
Epoch 7/30
1875/1875 [==============================] - 7s 4ms/step - loss: 0.2220 - accuracy: 0.9189
Epoch 8/30
1875/1875 [==============================] - 6s 3ms/step - loss: 0.2127 - accuracy: 0.9223
Epoch 9/30
1

In [24]:
# Load Data from Disk
from tensorflow.keras.models import load_model

loaded_model = load_model("model_autokeras", custom_objects=ak.CUSTOM_OBJECTS)
loaded_model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 28, 28, 1)]       0         
_________________________________________________________________
cast_to_float32 (CastToFloat (None, 28, 28, 1)         0         
_________________________________________________________________
normalization (Normalization (None, 28, 28, 1)         3         
_________________________________________________________________
conv2d (Conv2D)              (None, 26, 26, 32)        320       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 24, 24, 64)        18496     
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 12, 12, 64)        0         
_________________________________________________________________
dropout (Dropout)            (None, 12, 12, 64)        0     

In [25]:
# Predict with the best model.
# predicted_y = loaded_model.predict(tf.expand_dims(x_test, -1))
predicted_y = loaded_model.predict(x_test)
print(predicted_y)

[[8.9427614e-13 6.5947835e-17 5.9446016e-12 ... 2.6101841e-08
  3.3829370e-11 9.9999988e-01]
 [9.2816572e-06 8.4244688e-13 9.9986994e-01 ... 8.2195934e-16
  3.2444039e-10 1.3118513e-13]
 [1.4066848e-12 1.0000000e+00 1.1551899e-14 ... 6.2056744e-19
  3.3369182e-11 2.2314099e-18]
 ...
 [4.3457748e-11 1.4131873e-16 8.1874513e-10 ... 5.7741394e-19
  1.0000000e+00 8.2877892e-17]
 [4.5769040e-15 1.0000000e+00 5.4802587e-16 ... 2.3633386e-18
  7.7656649e-17 3.8750103e-17]
 [7.7618341e-07 4.2826365e-10 3.1859639e-07 ... 1.4149372e-03
  4.2980191e-05 1.4662389e-05]]


In [26]:
# Evaluate the best model with testing data.
print(clf.evaluate(x_test, y_test))

313/313 [==============================] - 1s 3ms/step - loss: 0.2438 - accuracy: 0.9214
[0.24384818971157074, 0.9214000105857849]
